## Universidad del Valle de Guatemala  
## Departamento de Ciencias de la Computación  
## Inteligencia Artificial - sección 10  

### Hoja de Trabajo 2

#### Nadissa Vela - 23764  
#### Cristian Túnchez - 231359

---


# Task 2 - Implementación Frozen Lake en Python

El agente debe cruzar un lago congelado representado por un grid de 4x4 donde:
- **S (Start):** Posición inicial
- **F (Frozen):** Camino seguro
- **H (Hole):** Agujero - Termina el juego con recompensa 0
- **G (Goal):** Meta - Recompensa +1

**Dinámica Estocástica:**  
El hielo es resbaladizo. Al tomar una acción, hay:
- 1/3 de probabilidad de moverse en la dirección deseada
- 1/3 de probabilidad de moverse a cada dirección perpendicular

Ejemplo: Si intenta ir Norte, puede terminar en Norte, Este u Oeste con igual probabilidad.

In [ ]:
# Importación de bibliotecas permitidas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

---
## Task 2.1 - Modelado del MDP

Implementaremos una clase `FrozenLakeMDP` que modele completamente el problema como un Proceso de Decisión de Markov.

### Componentes del MDP

Según la teoría, un MDP se define por:
1. **Estados (S):** Conjunto de todos los estados posibles
2. **Estado inicial (s_start):** Estado donde comienza el agente
3. **Acciones (Actions(s)):** Acciones disponibles en cada estado
4. **Función de transición (T(s, a, s')):** Probabilidad de llegar a s' al realizar acción a en estado s
5. **Función de recompensa (Reward(s, a, s')):** Recompensa recibida por la transición
6. **Estados finales (IsEnd(s)):** Indica si un estado es terminal
7. **Factor de descuento (γ):** Importancia de recompensas futuras (usaremos γ=0.9)

In [ ]:
class FrozenLakeMDP:
    """
    Implementación del problema Frozen Lake como un Markov Decision Process.
    
    Grid 4x4 con numeración de estados:
    [ 0  1  2  3]
    [ 4  5  6  7]
    [ 8  9 10 11]
    [12 13 14 15]
    
    Mapa del lago:
    S F F F
    F H F H
    F F F H
    H F F G
    
    Donde:
    S = Start (estado 0)
    F = Frozen (camino seguro)
    H = Hole (agujero, estado terminal)
    G = Goal (meta, estado terminal)
    """
    
    def __init__(self, gamma=0.9):
        """
        Inicializa el MDP del Frozen Lake.
        
        Args:
            gamma (float): Factor de descuento γ ∈ [0,1] para recompensas futuras
        """
        # Factor de descuento γ
        self.gamma = gamma
        
        # Dimensiones del grid
        self.rows = 4
        self.cols = 4
        self.n_states = self.rows * self.cols  # 16 estados (0-15)
        
        # Definición del mapa del lago (usando el layout estándar de Frozen Lake)
        # S: Start, F: Frozen, H: Hole, G: Goal
        self.lake_map = [
            ['S', 'F', 'F', 'F'],
            ['F', 'H', 'F', 'H'],
            ['F', 'F', 'F', 'H'],
            ['H', 'F', 'F', 'G']
        ]
        
        # Estado inicial
        self.start_state = 0  # Esquina superior izquierda
        
        # Estados terminales (Holes y Goal)
        # Holes: 5, 7, 11, 12
        # Goal: 15
        self.hole_states = {5, 7, 11, 12}
        self.goal_state = 15
        self.terminal_states = self.hole_states | {self.goal_state}
        
        # Definición de acciones: 0=Norte, 1=Sur, 2=Este, 3=Oeste
        self.actions = [0, 1, 2, 3]
        self.action_names = ['Norte', 'Sur', 'Este', 'Oeste']
        self.action_symbols = ['↑', '↓', '→', '←']
        
        # Movimientos correspondientes a cada acción
        # (delta_row, delta_col)
        self.action_deltas = {
            0: (-1, 0),  # Norte: arriba
            1: (1, 0),   # Sur: abajo
            2: (0, 1),   # Este: derecha
            3: (0, -1)   # Oeste: izquierda
        }
        
        # Precalcular las direcciones perpendiculares para cada acción
        # Para modelar la estocasticidad del hielo resbaladizo
        self.perpendicular_actions = {
            0: [2, 3],  # Norte -> perpendiculares son Este y Oeste
            1: [2, 3],  # Sur -> perpendiculares son Este y Oeste
            2: [0, 1],  # Este -> perpendiculares son Norte y Sur
            3: [0, 1]   # Oeste -> perpendiculares son Norte y Sur
        }
    
    def state_to_position(self, state):
        """
        Convierte un número de estado (0-15) a posición (row, col) en el grid.
        
        Args:
            state (int): Número de estado
            
        Returns:
            tuple: (row, col)
        """
        row = state // self.cols
        col = state % self.cols
        return row, col
    
    def position_to_state(self, row, col):
        """
        Convierte una posición (row, col) a número de estado.
        
        Args:
            row (int): Fila
            col (int): Columna
            
        Returns:
            int: Número de estado
        """
        return row * self.cols + col
    
    def is_valid_position(self, row, col):
        """
        Verifica si una posición está dentro del grid.
        
        Args:
            row (int): Fila
            col (int): Columna
            
        Returns:
            bool: True si la posición es válida
        """
        return 0 <= row < self.rows and 0 <= col < self.cols
    
    def is_terminal(self, state):
        """
        Verifica si un estado es terminal (IsEnd(s)).
        
        Estados terminales: Holes (5, 7, 11, 12) y Goal (15)
        
        Args:
            state (int): Número de estado
            
        Returns:
            bool: True si el estado es terminal
        """
        return state in self.terminal_states
    
    def get_next_state(self, state, action):
        """
        Calcula el estado resultante de tomar una acción desde un estado.
        No considera estocasticidad, solo calcula el resultado determinístico.
        
        Si el movimiento sale del grid, el agente permanece en el mismo estado.
        
        Args:
            state (int): Estado actual
            action (int): Acción a realizar (0=Norte, 1=Sur, 2=Este, 3=Oeste)
            
        Returns:
            int: Estado siguiente
        """
        # Si ya estamos en un estado terminal, nos quedamos ahí
        if self.is_terminal(state):
            return state
        
        # Calcular nueva posición
        row, col = self.state_to_position(state)
        delta_row, delta_col = self.action_deltas[action]
        new_row = row + delta_row
        new_col = col + delta_col
        
        # Verificar si la nueva posición es válida
        if self.is_valid_position(new_row, new_col):
            return self.position_to_state(new_row, new_col)
        else:
            # Si el movimiento sale del grid, permanecemos en el estado actual
            return state
    
    def get_transition_prob(self, state, action, next_state):
        """
        Calcula la probabilidad de transición T(s, a, s').
        
        Implementa la dinámica estocástica del hielo resbaladizo:
        - Probabilidad 1/3 de moverse en la dirección deseada
        - Probabilidad 1/3 de moverse en cada dirección perpendicular
        
        Fórmula del MDP:
        T(s, a, s') = P(s_{t+1} = s' | s_t = s, a_t = a)
        
        Args:
            state (int): Estado actual s
            action (int): Acción a
            next_state (int): Estado siguiente s'
            
        Returns:
            float: Probabilidad de transición T(s, a, s')
        """
        # Si estamos en un estado terminal, la probabilidad de quedarse es 1
        if self.is_terminal(state):
            return 1.0 if next_state == state else 0.0
        
        # Obtener los estados resultantes de la acción deseada y las perpendiculares
        intended_state = self.get_next_state(state, action)
        perp_actions = self.perpendicular_actions[action]
        perp_state1 = self.get_next_state(state, perp_actions[0])
        perp_state2 = self.get_next_state(state, perp_actions[1])
        
        # Contar cuántas veces aparece next_state en los posibles resultados
        # Cada aparición contribuye con probabilidad 1/3
        possible_states = [intended_state, perp_state1, perp_state2]
        count = possible_states.count(next_state)
        
        return count / 3.0
    
    def get_reward(self, state, action, next_state):
        """
        Función de recompensa Reward(s, a, s').
        
        Definición de recompensas:
        - Llegar al Goal (estado 15): +1
        - Llegar a un Hole: 0 (el juego termina sin recompensa)
        - Cualquier otra transición: 0
        
        Args:
            state (int): Estado actual s
            action (int): Acción a
            next_state (int): Estado siguiente s'
            
        Returns:
            float: Recompensa R(s, a, s')
        """
        # Recompensa de +1 solo al alcanzar el Goal
        if next_state == self.goal_state:
            return 1.0
        else:
            return 0.0
    
    def get_possible_next_states(self, state, action):
        """
        Obtiene todos los posibles estados siguientes al tomar una acción,
        junto con sus probabilidades de transición.
        
        Útil para la implementación eficiente de Value Iteration.
        
        Args:
            state (int): Estado actual
            action (int): Acción
            
        Returns:
            list: Lista de tuplas (next_state, probability, reward)
        """
        if self.is_terminal(state):
            return [(state, 1.0, 0.0)]
        
        # Calcular los tres posibles resultados
        intended_state = self.get_next_state(state, action)
        perp_actions = self.perpendicular_actions[action]
        perp_state1 = self.get_next_state(state, perp_actions[0])
        perp_state2 = self.get_next_state(state, perp_actions[1])
        
        # Agrupar estados idénticos y sumar sus probabilidades
        state_probs = {}
        for next_s in [intended_state, perp_state1, perp_state2]:
            if next_s in state_probs:
                state_probs[next_s] += 1/3.0
            else:
                state_probs[next_s] = 1/3.0
        
        # Crear lista de resultados con recompensas
        results = []
        for next_s, prob in state_probs.items():
            reward = self.get_reward(state, action, next_s)
            results.append((next_s, prob, reward))
        
        return results
    
    def visualize_grid(self):
        """
        Visualiza el mapa del Frozen Lake.
        """
        print("\nMapa del Frozen Lake (4x4):")
        print("="*30)
        for i, row in enumerate(self.lake_map):
            print(f"  {' '.join(row)}    (estados {i*4} a {i*4+3})")
        print("="*30)
        print("S = Start (inicio)")
        print("F = Frozen (camino seguro)")
        print("H = Hole (agujero - terminal)")
        print("G = Goal (meta - terminal, recompensa +1)")
        print(f"\nEstados terminales: {sorted(self.terminal_states)}")
        print(f"Factor de descuento γ = {self.gamma}")

### Prueba del Modelado del MDP

Verificamos que la función de transición T(s, a, s') y la función de recompensa R(s, a, s') estén correctamente implementadas.

In [ ]:
# Crear instancia del MDP
mdp = FrozenLakeMDP(gamma=0.9)

# Visualizar el mapa
mdp.visualize_grid()

In [ ]:
# Ejemplo de función de transición T(s, a, s')
print("\nEjemplo de Función de Transición T(s, a, s')")
print("="*50)

# Desde el estado inicial (0), intentamos ir al Sur (acción 1)
state = 0
action = 1  # Sur

print(f"\nDesde estado {state}, acción: {mdp.action_names[action]} ({mdp.action_symbols[action]})")
print("\nDinámica estocástica del hielo resbaladizo:")
print(f"- Dirección deseada (Sur): estado {mdp.get_next_state(state, 1)}")
print(f"- Dirección perpendicular 1 (Este): estado {mdp.get_next_state(state, 2)}")
print(f"- Dirección perpendicular 2 (Oeste): estado {mdp.get_next_state(state, 3)}")

print("\nProbabilidades de transición T(s, a, s'):")
for s_prime in range(mdp.n_states):
    prob = mdp.get_transition_prob(state, action, s_prime)
    if prob > 0:
        print(f"  T({state}, {mdp.action_names[action]}, {s_prime}) = {prob:.3f}")

# Verificar que las probabilidades suman 1
total_prob = sum(mdp.get_transition_prob(state, action, s_prime) 
                 for s_prime in range(mdp.n_states))
print(f"\nSuma de probabilidades: {total_prob:.3f} (debe ser 1.0)")

In [ ]:
# Ejemplo de función de recompensa R(s, a, s')
print("\nEjemplo de Función de Recompensa R(s, a, s')")
print("="*50)

# Transición al Goal
print(f"\nLlegar al Goal (estado {mdp.goal_state}):")
print(f"  R(14, Este, 15) = {mdp.get_reward(14, 2, 15)}")

# Transición a un Hole
print(f"\nLlegar a un Hole (estado 5):")
print(f"  R(1, Sur, 5) = {mdp.get_reward(1, 1, 5)}")

# Transición normal
print(f"\nTransición normal (sin llegar a terminal):")
print(f"  R(0, Este, 1) = {mdp.get_reward(0, 2, 1)}")

---
## Task 2.2 - Algoritmo de Iteración de Valores (Value Iteration)

Implementaremos el algoritmo de **Value Iteration** basado en las diapositivas de la presentación.

### Ecuación de Bellman para Value Iteration

La actualización iterativa se realiza mediante:

$$V_{k+1}(s) = \max_{a \in Actions(s)} \sum_{s'} T(s, a, s') [Reward(s, a, s') + \gamma V_k(s')]$$

Donde:
- $V_k(s)$ es el valor del estado $s$ en la iteración $k$
- $T(s, a, s')$ es la probabilidad de transición
- $Reward(s, a, s')$ es la recompensa inmediata
- $\gamma$ es el factor de descuento (0.9 en nuestro caso)

### Extracción de Política Óptima

Una vez que $V$ ha convergido a $V_{opt}$, la política óptima se extrae mediante:

$$\pi_{opt}(s) = \arg\max_{a} Q_{opt}(s, a)$$

Donde el Q-value óptimo es:

$$Q_{opt}(s, a) = \sum_{s'} T(s, a, s') [R(s, a, s') + \gamma V_{opt}(s')]$$

In [ ]:
class ValueIteration:
    """
    Implementación del algoritmo de Value Iteration para resolver un MDP.
    
    Basado en la Ecuación de Bellman:
    V_{k+1}(s) = max_a Σ_{s'} T(s,a,s') [R(s,a,s') + γ V_k(s')]
    """
    
    def __init__(self, mdp, epsilon=1e-6, max_iterations=1000):
        """
        Inicializa el algoritmo de Value Iteration.
        
        Args:
            mdp (FrozenLakeMDP): El MDP a resolver
            epsilon (float): Criterio de convergencia (diferencia máxima entre iteraciones)
            max_iterations (int): Número máximo de iteraciones para evitar loops infinitos
        """
        self.mdp = mdp
        self.epsilon = epsilon
        self.max_iterations = max_iterations
        
        # Inicialización: V_0(s) = 0 para todos los estados
        # Como indica la presentación, comenzamos con valores de utilidad cero
        self.V = np.zeros(mdp.n_states)
        
        # Política óptima (se calculará después de la convergencia)
        self.policy = None
        
        # Historial de valores para análisis de convergencia
        self.value_history = []
        
        # Número de iteraciones realizadas
        self.iterations = 0
    
    def compute_q_value(self, state, action):
        """
        Calcula el Q-value para un par estado-acción.
        
        Fórmula (de la presentación):
        Q(s, a) = Σ_{s'} T(s, a, s') [R(s, a, s') + γ V(s')]
        
        Esta es la utilidad esperada de tomar la acción 'a' en el estado 's'
        y luego seguir la política óptima.
        
        Args:
            state (int): Estado actual s
            action (int): Acción a
            
        Returns:
            float: Q-value Q(s, a)
        """
        # Si es un estado terminal, el Q-value es 0 (no hay acciones útiles)
        if self.mdp.is_terminal(state):
            return 0.0
        
        q_value = 0.0
        
        # Suma sobre todos los posibles estados siguientes s'
        # Implementación vectorial de la fórmula de Bellman
        transitions = self.mdp.get_possible_next_states(state, action)
        
        for next_state, prob, reward in transitions:
            # T(s, a, s') * [R(s, a, s') + γ * V(s')]
            q_value += prob * (reward + self.mdp.gamma * self.V[next_state])
        
        return q_value
    
    def value_iteration_step(self):
        """
        Realiza una iteración del algoritmo de Value Iteration.
        
        Actualiza V_{k+1}(s) para todos los estados usando la Ecuación de Bellman:
        V_{k+1}(s) = max_a Σ_{s'} T(s,a,s') [R(s,a,s') + γ V_k(s')]
        
        Returns:
            float: Diferencia máxima |V_{k+1}(s) - V_k(s)| entre iteraciones
        """
        # Crear un nuevo array para V_{k+1}
        V_new = np.zeros(self.mdp.n_states)
        
        # Para cada estado s
        for state in range(self.mdp.n_states):
            if self.mdp.is_terminal(state):
                # Los estados terminales mantienen valor 0
                V_new[state] = 0.0
            else:
                # Calcular Q(s, a) para cada acción y tomar el máximo
                # V(s) = max_a Q(s, a)
                q_values = [self.compute_q_value(state, action) 
                           for action in self.mdp.actions]
                V_new[state] = max(q_values)
        
        # Calcular la diferencia máxima (criterio de convergencia)
        max_diff = np.max(np.abs(V_new - self.V))
        
        # Actualizar los valores
        self.V = V_new
        
        return max_diff
    
    def solve(self, verbose=True):
        """
        Ejecuta el algoritmo de Value Iteration hasta la convergencia.
        
        El algoritmo itera hasta que:
        max_s |V_{k+1}(s) - V_k(s)| < ε
        
        Args:
            verbose (bool): Si True, imprime información de progreso
            
        Returns:
            tuple: (V_optimal, numero_iteraciones)
        """
        if verbose:
            print("Iniciando Value Iteration...")
            print(f"Criterio de convergencia: ε = {self.epsilon}")
            print(f"Factor de descuento: γ = {self.mdp.gamma}")
            print("\n" + "="*60)
        
        for iteration in range(self.max_iterations):
            # Guardar historial para análisis
            self.value_history.append(self.V.copy())
            
            # Realizar una iteración
            max_diff = self.value_iteration_step()
            self.iterations = iteration + 1
            
            # Mostrar progreso cada 10 iteraciones
            if verbose and (iteration + 1) % 10 == 0:
                print(f"Iteración {iteration + 1:3d}: max_diff = {max_diff:.6f}")
            
            # Verificar convergencia
            if max_diff < self.epsilon:
                if verbose:
                    print("="*60)
                    print(f"\nConvergencia alcanzada en {iteration + 1} iteraciones!")
                    print(f"Diferencia final: {max_diff:.8f} < ε = {self.epsilon}")
                break
        else:
            if verbose:
                print(f"\nAdvertencia: Se alcanzó el máximo de iteraciones ({self.max_iterations})")
        
        return self.V, self.iterations
    
    def extract_policy(self):
        """
        Extrae la política óptima π* a partir de los valores óptimos V*.
        
        Fórmula (de la presentación):
        π_{opt}(s) = argmax_a Q_{opt}(s, a)
        
        donde:
        Q_{opt}(s, a) = Σ_{s'} T(s,a,s') [R(s,a,s') + γ V_{opt}(s')]
        
        Returns:
            numpy.ndarray: Array de tamaño n_states con la acción óptima para cada estado
        """
        policy = np.zeros(self.mdp.n_states, dtype=int)
        
        for state in range(self.mdp.n_states):
            if self.mdp.is_terminal(state):
                # En estados terminales, la acción no importa (ponemos 0 por convención)
                policy[state] = 0
            else:
                # Calcular Q(s, a) para cada acción
                q_values = [self.compute_q_value(state, action) 
                           for action in self.mdp.actions]
                
                # Seleccionar la acción que maximiza el Q-value
                # π*(s) = argmax_a Q(s, a)
                policy[state] = np.argmax(q_values)
        
        self.policy = policy
        return policy
    
    def print_values_grid(self):
        """
        Imprime los valores V(s) en formato de grid 4x4.
        """
        print("\nValores V(s) óptimos (formato grid 4x4):")
        print("="*50)
        for row in range(self.mdp.rows):
            row_values = []
            for col in range(self.mdp.cols):
                state = self.mdp.position_to_state(row, col)
                row_values.append(f"{self.V[state]:6.3f}")
            print("  " + "  ".join(row_values))
        print("="*50)
    
    def print_policy_grid(self):
        """
        Imprime la política óptima π*(s) en formato de grid 4x4 con flechas.
        """
        if self.policy is None:
            print("Error: Primero debe extraer la política con extract_policy()")
            return
        
        print("\nPolítica Óptima π*(s) (formato grid 4x4):")
        print("="*50)
        for row in range(self.mdp.rows):
            row_symbols = []
            for col in range(self.mdp.cols):
                state = self.mdp.position_to_state(row, col)
                
                # Mostrar el tipo de celda para estados terminales
                if state in self.mdp.hole_states:
                    symbol = "H"
                elif state == self.mdp.goal_state:
                    symbol = "G"
                else:
                    # Mostrar la flecha de la acción óptima
                    action = self.policy[state]
                    symbol = self.mdp.action_symbols[action]
                
                row_symbols.append(f"  {symbol}  ")
            print("".join(row_symbols))
        print("="*50)
        print("Leyenda: ↑=Norte  ↓=Sur  →=Este  ←=Oeste  H=Hole  G=Goal")

### Ejecución del Algoritmo de Value Iteration

Ahora ejecutaremos el algoritmo para encontrar la política óptima.

In [ ]:
# Crear instancia del algoritmo de Value Iteration
vi = ValueIteration(mdp, epsilon=1e-6, max_iterations=1000)

# Ejecutar el algoritmo hasta convergencia
V_optimal, num_iterations = vi.solve(verbose=True)

In [ ]:
# Mostrar los valores óptimos V*(s)
vi.print_values_grid()

In [ ]:
# Extraer y mostrar la política óptima π*(s)
optimal_policy = vi.extract_policy()
vi.print_policy_grid()

---
## Visualizaciones

### 1. Mapa de Calor de los Valores V(s)

Este mapa muestra qué celdas son las más valiosas. Los estados con valores más altos son aquellos desde los cuales es más probable alcanzar el Goal con buenas recompensas.

In [ ]:
def plot_value_heatmap(vi):
    """
    Crea un mapa de calor de los valores V(s) óptimos.
    
    Los colores más cálidos (rojos/naranjas) indican estados más valiosos,
    es decir, estados desde los cuales se puede alcanzar el Goal con mayor
    utilidad esperada.
    """
    # Convertir el vector de valores a matriz 4x4
    V_grid = vi.V.reshape((vi.mdp.rows, vi.mdp.cols))
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Crear mapa de calor
    im = ax.imshow(V_grid, cmap='YlOrRd', aspect='equal')
    
    # Agregar barra de color
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Valor V(s)', rotation=270, labelpad=20, fontsize=12)
    
    # Configurar ejes
    ax.set_xticks(np.arange(vi.mdp.cols))
    ax.set_yticks(np.arange(vi.mdp.rows))
    ax.set_xticklabels(np.arange(vi.mdp.cols))
    ax.set_yticklabels(np.arange(vi.mdp.rows))
    
    # Agregar grid
    ax.set_xticks(np.arange(vi.mdp.cols + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(vi.mdp.rows + 1) - 0.5, minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=2)
    
    # Agregar valores numéricos y etiquetas de celda
    for i in range(vi.mdp.rows):
        for j in range(vi.mdp.cols):
            state = vi.mdp.position_to_state(i, j)
            cell_type = vi.mdp.lake_map[i][j]
            
            # Determinar color del texto según el valor del fondo
            text_color = 'white' if V_grid[i, j] > 0.3 else 'black'
            
            # Agregar etiqueta de tipo de celda
            ax.text(j, i - 0.25, cell_type, ha='center', va='center',
                   color=text_color, fontsize=14, fontweight='bold')
            
            # Agregar valor numérico
            ax.text(j, i + 0.1, f'{V_grid[i, j]:.3f}', ha='center', va='center',
                   color=text_color, fontsize=10)
            
            # Agregar número de estado
            ax.text(j, i + 0.35, f'(s={state})', ha='center', va='center',
                   color=text_color, fontsize=8, style='italic')
    
    ax.set_title('Mapa de Calor de Valores V(s) Óptimos\n' + 
                'Los estados más valiosos están en rojo/naranja',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Columna', fontsize=12)
    ax.set_ylabel('Fila', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Análisis de los estados más valiosos
    print("\nAnálisis de Estados Más Valiosos:")
    print("="*60)
    
    # Ordenar estados por valor
    state_values = [(s, vi.V[s]) for s in range(vi.mdp.n_states)]
    state_values.sort(key=lambda x: x[1], reverse=True)
    
    print("\nTop 5 estados más valiosos:")
    for idx, (state, value) in enumerate(state_values[:5], 1):
        row, col = vi.mdp.state_to_position(state)
        cell_type = vi.mdp.lake_map[row][col]
        print(f"{idx}. Estado {state} (fila {row}, col {col}, tipo '{cell_type}'): V = {value:.4f}")
    
    print("\n¿Por qué estos estados son valiosos?")
    print("Los estados cerca del Goal (G) tienen valores más altos porque:")
    print("- Están más cerca de la recompensa de +1")
    print("- Con γ=0.9, las recompensas futuras cercanas valen más que las lejanas")
    print("- Tienen menor riesgo de caer en un Hole antes de alcanzar el Goal")

# Generar el mapa de calor
plot_value_heatmap(vi)

### 2. Visualización de la Política Óptima con Flechas

In [ ]:
def plot_optimal_policy(vi):
    """
    Visualiza la política óptima π*(s) mostrando flechas para cada estado.
    
    Las flechas indican la dirección óptima a seguir desde cada estado.
    Los estados terminales (Holes y Goal) se marcan de forma especial.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Crear una matriz de colores para el fondo
    background = np.ones((vi.mdp.rows, vi.mdp.cols))
    
    # Marcar estados especiales
    for i in range(vi.mdp.rows):
        for j in range(vi.mdp.cols):
            state = vi.mdp.position_to_state(i, j)
            if state in vi.mdp.hole_states:
                background[i, j] = 0.3  # Holes en gris oscuro
            elif state == vi.mdp.goal_state:
                background[i, j] = 0.7  # Goal en verde claro
    
    # Mostrar fondo
    ax.imshow(background, cmap='RdYlGn', aspect='equal', alpha=0.5, vmin=0, vmax=1)
    
    # Configurar grid
    ax.set_xticks(np.arange(vi.mdp.cols))
    ax.set_yticks(np.arange(vi.mdp.rows))
    ax.set_xticklabels(np.arange(vi.mdp.cols))
    ax.set_yticklabels(np.arange(vi.mdp.rows))
    
    ax.set_xticks(np.arange(vi.mdp.cols + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(vi.mdp.rows + 1) - 0.5, minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    
    # Mapeo de acciones a vectores de dirección para las flechas
    action_to_arrow = {
        0: (0, 0.3),    # Norte: arriba
        1: (0, -0.3),   # Sur: abajo
        2: (0.3, 0),    # Este: derecha
        3: (-0.3, 0)    # Oeste: izquierda
    }
    
    # Dibujar flechas y etiquetas
    for i in range(vi.mdp.rows):
        for j in range(vi.mdp.cols):
            state = vi.mdp.position_to_state(i, j)
            cell_type = vi.mdp.lake_map[i][j]
            
            if state in vi.mdp.hole_states:
                # Marcar Holes
                ax.text(j, i, 'H\nHole', ha='center', va='center',
                       fontsize=14, fontweight='bold', color='white')
            elif state == vi.mdp.goal_state:
                # Marcar Goal
                ax.text(j, i, 'G\nGoal', ha='center', va='center',
                       fontsize=14, fontweight='bold', color='darkgreen')
            else:
                # Dibujar flecha de la acción óptima
                action = vi.policy[state]
                dx, dy = action_to_arrow[action]
                
                ax.arrow(j, i, dx, dy, head_width=0.15, head_length=0.1,
                        fc='blue', ec='blue', linewidth=2)
                
                # Agregar etiqueta de acción
                action_name = vi.mdp.action_names[action]
                ax.text(j, i + 0.45, action_name[0], ha='center', va='center',
                       fontsize=9, style='italic', color='darkblue')
            
            # Agregar número de estado en la esquina
            ax.text(j - 0.4, i - 0.4, str(state), ha='left', va='top',
                   fontsize=8, color='black', bbox=dict(boxstyle='round,pad=0.3',
                   facecolor='white', alpha=0.7))
    
    ax.set_title('Política Óptima π*(s) - Acción Óptima para cada Estado\n' +
                'Las flechas indican la dirección óptima a seguir',
                fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Columna', fontsize=12)
    ax.set_ylabel('Fila', fontsize=12)
    
    # Agregar leyenda
    legend_elements = [
        mpatches.Patch(facecolor='lightgreen', edgecolor='black', label='Goal (G)'),
        mpatches.Patch(facecolor='gray', edgecolor='black', label='Hole (H)'),
        mpatches.Patch(facecolor='lightyellow', edgecolor='black', label='Frozen (F)'),
        mpatches.FancyArrow(0, 0, 0.3, 0, color='blue', label='Acción óptima')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))
    
    plt.tight_layout()
    plt.show()

# Generar la visualización de la política
plot_optimal_policy(vi)

### 3. Convergencia del Algoritmo

In [ ]:
def plot_convergence(vi):
    """
    Visualiza la convergencia del algoritmo de Value Iteration.
    
    Muestra cómo los valores V(s) evolucionan a lo largo de las iteraciones
    hasta alcanzar la convergencia.
    """
    history = np.array(vi.value_history)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Gráfico 1: Evolución de valores para algunos estados clave
    estados_clave = [0, 10, 14, 15]  # Start, centro, cerca de Goal, Goal
    estados_nombres = ['Estado 0 (Start)', 'Estado 10 (Centro)', 
                      'Estado 14 (Cerca Goal)', 'Estado 15 (Goal)']
    
    for estado, nombre in zip(estados_clave, estados_nombres):
        valores = history[:, estado]
        ax1.plot(valores, marker='o', markersize=3, label=nombre, linewidth=2)
    
    ax1.set_xlabel('Iteración', fontsize=12)
    ax1.set_ylabel('Valor V(s)', fontsize=12)
    ax1.set_title('Convergencia de Valores V(s) por Iteración', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Gráfico 2: Diferencia máxima entre iteraciones
    if len(history) > 1:
        max_diffs = []
        for i in range(1, len(history)):
            diff = np.max(np.abs(history[i] - history[i-1]))
            max_diffs.append(diff)
        
        ax2.plot(max_diffs, color='red', linewidth=2, marker='o', markersize=3)
        ax2.axhline(y=vi.epsilon, color='green', linestyle='--', linewidth=2, 
                   label=f'Umbral ε = {vi.epsilon}')
        ax2.set_xlabel('Iteración', fontsize=12)
        ax2.set_ylabel('max |V_{k+1}(s) - V_k(s)|', fontsize=12)
        ax2.set_title('Criterio de Convergencia\n' +
                     'Diferencia Máxima entre Iteraciones', 
                     fontsize=13, fontweight='bold')
        ax2.set_yscale('log')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nAnálisis de Convergencia:")
    print("="*60)
    print(f"Número total de iteraciones: {vi.iterations}")
    print(f"Criterio de convergencia (ε): {vi.epsilon}")
    print(f"\nLa convergencia se alcanza cuando:")
    print(f"  max_s |V_{{k+1}}(s) - V_k(s)| < ε")
    print(f"\nEsto significa que los valores han dejado de cambiar significativamente,")
    print(f"indicando que hemos encontrado V* (los valores óptimos).")

# Generar la visualización de convergencia
plot_convergence(vi)

---
## Análisis de Resultados

### Interpretación de los Valores V(s)

Los valores V(s) representan la **utilidad esperada** de comenzar en el estado s y seguir la política óptima. Observamos que:

1. **Estados cerca del Goal tienen valores más altos:** Esto se debe a que:
   - Están más cerca de la recompensa de +1
   - Con γ=0.9, las recompensas futuras se descuentan, por lo que alcanzar el Goal en menos pasos vale más
   
2. **Estados cerca de Holes tienen valores más bajos:** El riesgo de caer en un agujero reduce la utilidad esperada.

3. **La estocasticidad afecta los valores:** Debido a que el hielo es resbaladizo (solo 1/3 de probabilidad de moverse en la dirección deseada), algunos estados que parecen cercanos al Goal pueden tener valores menores si están rodeados de Holes.

### Interpretación de la Política Óptima π*(s)

La política óptima nos dice qué acción tomar en cada estado para maximizar la utilidad esperada. Las flechas muestran el camino óptimo considerando:

1. **La estocasticidad del movimiento:** La política no siempre apunta directamente al Goal porque debe considerar el riesgo de resbalar hacia Holes.

2. **Equilibrio entre riesgo y recompensa:** En algunos estados, la política óptima puede preferir un camino más largo pero más seguro en lugar de un atajo riesgoso.

### Relación con las Fórmulas de Bellman

Los resultados obtenidos son consecuencia directa de la **Ecuación de Bellman**:

$$V_{k+1}(s) = \max_{a \in Actions(s)} \sum_{s'} T(s, a, s') [Reward(s, a, s') + \gamma V_k(s')]$$

Esta ecuación:
- Considera todas las transiciones posibles desde el estado actual
- Pondera cada transición por su probabilidad T(s, a, s')
- Suma la recompensa inmediata más el valor futuro descontado
- Elige la acción que maximiza esta suma

El factor de descuento γ=0.9 significa que una recompensa de +1 en el siguiente paso vale 0.9, en dos pasos vale 0.9² = 0.81, etc. Esto favorece políticas que alcancen el Goal rápidamente.

---
## Simulación de un Episodio

Para validar nuestra política óptima, simularemos algunos episodios siguiendo π*(s).

In [ ]:
def simulate_episode(mdp, policy, max_steps=100, seed=None):
    """
    Simula un episodio en el Frozen Lake siguiendo una política dada.
    
    Args:
        mdp (FrozenLakeMDP): El MDP
        policy (numpy.ndarray): Política a seguir
        max_steps (int): Número máximo de pasos para evitar loops infinitos
        seed (int): Semilla para reproducibilidad
        
    Returns:
        tuple: (trayectoria, recompensa_total, exito)
    """
    if seed is not None:
        np.random.seed(seed)
    
    state = mdp.start_state
    trajectory = [state]
    total_reward = 0.0
    
    for step in range(max_steps):
        if mdp.is_terminal(state):
            break
        
        # Seleccionar acción según la política
        action = policy[state]
        
        # Obtener transiciones posibles con sus probabilidades
        transitions = mdp.get_possible_next_states(state, action)
        
        # Samplear el siguiente estado según las probabilidades
        next_states = [t[0] for t in transitions]
        probs = [t[1] for t in transitions]
        rewards = [t[2] for t in transitions]
        
        idx = np.random.choice(len(next_states), p=probs)
        next_state = next_states[idx]
        reward = rewards[idx]
        
        # Actualizar
        total_reward += reward
        state = next_state
        trajectory.append(state)
    
    success = (state == mdp.goal_state)
    
    return trajectory, total_reward, success

def visualize_trajectory(mdp, trajectory):
    """
    Visualiza una trayectoria en el grid.
    """
    print("\nTrayectoria del Agente:")
    print("="*60)
    
    for step, state in enumerate(trajectory):
        row, col = mdp.state_to_position(state)
        cell_type = mdp.lake_map[row][col]
        print(f"Paso {step}: Estado {state} (fila {row}, col {col}) - Tipo: {cell_type}")
    
    print("="*60)

# Simular varios episodios
print("Simulación de Episodios con la Política Óptima")
print("="*60)

num_episodes = 5
successes = 0

for episode in range(num_episodes):
    trajectory, reward, success = simulate_episode(mdp, optimal_policy, seed=episode)
    
    print(f"\n--- Episodio {episode + 1} ---")
    print(f"Pasos totales: {len(trajectory) - 1}")
    print(f"Recompensa total: {reward}")
    print(f"¿Alcanzó el Goal?: {'Sí' if success else 'No'}")
    
    if success:
        successes += 1
    
    visualize_trajectory(mdp, trajectory)

print(f"\n{'='*60}")
print(f"Tasa de éxito: {successes}/{num_episodes} = {successes/num_episodes*100:.1f}%")
print(f"\nNota: Debido a la naturaleza estocástica del hielo resbaladizo,")
print(f"no todos los episodios alcanzarán el Goal, incluso con la política óptima.")

---
## Conclusiones

### Task 2.1 - Modelado del MDP

Hemos implementado exitosamente el Frozen Lake como un Markov Decision Process, definiendo:

1. **Estados (S):** 16 estados numerados de 0 a 15 en un grid 4x4
2. **Acciones (A):** Norte, Sur, Este, Oeste
3. **Función de transición T(s, a, s'):** Captura la estocasticidad del hielo resbaladizo con probabilidades 1/3
4. **Función de recompensa R(s, a, s'):** +1 al alcanzar el Goal, 0 en otros casos
5. **Estados terminales:** Holes (5, 7, 11, 12) y Goal (15)

### Task 2.2 - Value Iteration

Hemos implementado el algoritmo de Value Iteration siguiendo la Ecuación de Bellman:

$$V_{k+1}(s) = \max_{a} \sum_{s'} T(s, a, s') [R(s, a, s') + \gamma V_k(s')]$$

El algoritmo:
- Converge exitosamente a la solución óptima V*
- Extrae la política óptima π* mediante argmax de los Q-values
- Utiliza γ=0.9 como factor de descuento

### Insights Principales

1. **La estocasticidad complica la planificación:** El hielo resbaladizo requiere que la política considere movimientos no deseados, resultando en caminos que no siempre apuntan directamente al Goal.

2. **El factor de descuento importa:** Con γ=0.9, el agente prefiere alcanzar el Goal rápidamente, ya que recompensas futuras valen menos.

3. **Value Iteration es eficiente:** El algoritmo converge en pocas iteraciones, calculando tanto los valores óptimos como la política óptima.

4. **Los valores V(s) reflejan utilidad esperada:** Estados más valiosos son aquellos desde los cuales es más probable alcanzar el Goal con buena recompensa.

### Conexión con la Teoría

Todo el desarrollo se basa rigurosamente en las fórmulas presentadas en clase:

- La Ecuación de Bellman para actualización de valores
- La definición formal de un MDP con sus componentes
- El concepto de política óptima y Q-values
- El factor de descuento γ para valorar recompensas futuras

Esta implementación demuestra cómo la teoría de MDPs se aplica a problemas reales con incertidumbre, proporcionando una solución sistemática y óptima.